## References
https://www.medi-08-data-06.work/entry/regression_stan

In [2]:
import numpy as np
import numpy.random as rd
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation as ani
import matplotlib.cm as cm
#import seaborn as sns
#sns.set(style="whitegrid", palette="muted", color_codes=True)

#from tabulate import tabulate
from time import time

import pystan
from pystan import StanModel

import statsmodels.api as sm
import statsmodels.formula.api as smf

### data

In [3]:
data = pd.read_csv("./data.csv")
col_name = data.columns
data

,Region,Type,1st Size,2nd Size,1st mean,2nd mean,1st std,2nd std
0,A,1,55,51,151.36,157.27,2.94,2.98
1,B,1,53,49,151.56,156.83,3.07,3.14
2,C,0,55,53,152.22,157.08,3.20,3.21
3,D,1,53,52,153.09,156.00,2.65,2.64
4,E,1,58,55,153.22,157.24,3.07,3.03
5,F,0,55,53,153.31,157.22,3.10,3.13
6,G,0,58,53,152.98,157.81,2.49,2.45
7,H,0,59,57,153.27,158.95,3.08,3.06
8,I,1,56,51,152.67,156.82,2.82,2.92
9,J,0,56,50,155.37,161.71,3.10,3.21


In [4]:
# data preparation
Y = np.hstack([data["1st mean"], data["2nd mean"]])
Age = np.hstack([np.zeros(10), np.ones(10)])
X = np.hstack([data["Type"], data["Type"]])
X = np.hstack([X.reshape(-1,1), Age.reshape(-1,1)])
X = pd.DataFrame(X)

data_model1 = pd.concat([X,pd.DataFrame(Y)], axis=1)
data_model1.columns = ["Type", "Age", "meanY"]
data_model1

,Type,Age,meanY
0,1.0,0.0,151.36
1,1.0,0.0,151.56
2,0.0,0.0,152.22
3,1.0,0.0,153.09
4,1.0,0.0,153.22
5,0.0,0.0,153.31
6,0.0,0.0,152.98
7,0.0,0.0,153.27
8,1.0,0.0,152.67
9,0.0,0.0,155.37


In [11]:
formula = "meanY ~  1 + Age + Age:Type"
link = sm.genmod.families.links.identity
family = sm.families.Gaussian(link=link)

mod = smf.glm(formula=formula, data=data_model1, family=family )
result = mod.fit() 
print(result.summary())

print(result.aic)

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  meanY   No. Observations:                   20
Model:                            GLM   Df Residuals:                       17
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                          1.5827
Method:                          IRLS   Log-Likelihood:                -31.345
Date:                Mon, 22 Jun 2020   Deviance:                       26.907
Time:                        18:56:01   Pearson chi2:                     26.9
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    152.9050      0.398    384.341      0.0

C:\Users\chiak\Anaconda3\envs\stan\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: 'GLMResults' object has no attribute 'coef'